# Flight Search Agent - Agent Catalog + Couchbase Tutorial

## Overview

This tutorial demonstrates how to build a flight search agent using:
- **Agent Catalog**: Tool registration and agent framework
- **Couchbase**: Vector database for semantic search and data storage
- **LangGraph**: Agent workflow orchestration
- **OpenAI**: Language models and embeddings

### What You'll Learn
1. Setting up Couchbase infrastructure with vector search
2. Registering tools with Agent Catalog
3. Building intelligent parameter mapping
4. Creating a ReAct agent with LangGraph
5. Implementing flight search, booking, and policy queries

### Architecture Overview
```
User Query → LangGraph Agent → Agent Catalog Tools → Couchbase Database
                ↓
Parameter Mapping → Tool Execution → Vector Search → Response Generation
```

## Section 1: Environment Setup and Dependencies

First, let's install the required dependencies and set up our environment.

In [1]:
# Install required packages (uncomment if not already installed)
# !pip install langchain langchain-openai langgraph couchbase python-dotenv langchain-couchbase
# !pip install agentc-core agentc-langgraph  # These should be installed from local paths

In [2]:
# Import required libraries
import getpass
import json
import logging
import os
import sys
import time
from datetime import timedelta

import agentc
import agentc_langgraph.agent
import agentc_langgraph.graph
import dotenv
import langchain_core.messages
import langchain_core.runnables
import langchain_openai.chat_models
import langgraph.graph
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from langchain_couchbase.vectorstores import CouchbaseSearchVectorStore
from langchain_openai import OpenAIEmbeddings

# Load environment variables
dotenv.load_dotenv(override=True)

# Setup logging
logging.basicConfig(
    level=logging.DEBUG, 
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

print("All dependencies imported successfully!")

All dependencies imported successfully!


### Environment Variables Setup

Configure the required environment variables for Couchbase and OpenAI connections.

In [3]:
def _set_if_undefined(var: str):
    if os.environ.get(var) is None:
        os.environ[var] = getpass.getpass(f"Please provide your {var}: ")

def setup_environment():
    """Setup required environment variables with defaults."""
    required_vars = ["OPENAI_API_KEY", "CB_CONN_STRING", "CB_USERNAME", "CB_PASSWORD", "CB_BUCKET"]
    for var in required_vars:
        _set_if_undefined(var)

    defaults = {
        "CB_CONN_STRING": "couchbase://localhost",
        "CB_USERNAME": "Administrator",
        "CB_PASSWORD": "password",
        "CB_BUCKET": "vector-search-testing",
    }

    for key, default_value in defaults.items():
        if not os.environ.get(key):
            os.environ[key] = input(f"Enter {key} (default: {default_value}): ") or default_value

    os.environ["INDEX_NAME"] = os.getenv("INDEX_NAME", "vector_search_agentcatalog")
    os.environ["SCOPE_NAME"] = os.getenv("SCOPE_NAME", "shared")
    os.environ["COLLECTION_NAME"] = os.getenv("COLLECTION_NAME", "agentcatalog")

# Setup environment
setup_environment()
print("Environment variables configured successfully!")
print(f"Using bucket: {os.environ['CB_BUCKET']}")

Environment variables configured successfully!
Using bucket: vector-search-testing


## Section 2: Couchbase Infrastructure Setup

Now let's establish our Couchbase connection and set up the necessary infrastructure.

In [4]:
def setup_couchbase_connection():
    """Setup Couchbase cluster connection."""
    try:
        auth = PasswordAuthenticator(os.environ["CB_USERNAME"], os.environ["CB_PASSWORD"])
        options = ClusterOptions(auth)
        cluster = Cluster(os.environ["CB_CONN_STRING"], options)
        cluster.wait_until_ready(timedelta(seconds=10))
        logger.info("Successfully connected to Couchbase")
        return cluster
    except Exception as e:
        raise ConnectionError(f"Failed to connect to Couchbase: {e!s}")

# Establish connection
cluster = setup_couchbase_connection()

2025-07-07 12:14:42,820 - __main__ - INFO - Successfully connected to Couchbase


In [5]:
def setup_collection(cluster, bucket_name, scope_name, collection_name):
    """Setup Couchbase bucket, scope and collection."""
    try:
        try:
            bucket = cluster.bucket(bucket_name)
            logger.info(f"Bucket '{bucket_name}' exists")
        except Exception:
            logger.info(f"Creating bucket '{bucket_name}'...")
            bucket_settings = CreateBucketSettings(
                name=bucket_name,
                bucket_type="couchbase",
                ram_quota_mb=1024,
                flush_enabled=True,
                num_replicas=0,
            )
            cluster.buckets().create_bucket(bucket_settings)
            time.sleep(5)
            bucket = cluster.bucket(bucket_name)
            logger.info(f"Bucket '{bucket_name}' created successfully")

        bucket_manager = bucket.collections()

        scopes = bucket_manager.get_all_scopes()
        scope_exists = any(scope.name == scope_name for scope in scopes)

        if not scope_exists and scope_name != "_default":
            logger.info(f"Creating scope '{scope_name}'...")
            bucket_manager.create_scope(scope_name)
            logger.info(f"Scope '{scope_name}' created successfully")

        collections = bucket_manager.get_all_scopes()
        collection_exists = any(
            scope.name == scope_name and collection_name in [col.name for col in scope.collections]
            for scope in collections
        )

        if not collection_exists:
            logger.info(f"Creating collection '{collection_name}'...")
            bucket_manager.create_collection(scope_name, collection_name)
            logger.info(f"Collection '{collection_name}' created successfully")

        collection = bucket.scope(scope_name).collection(collection_name)
        time.sleep(3)

        try:
            cluster.query(
                f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{bucket_name}`.`{scope_name}`.`{collection_name}`"
            ).execute()
            logger.info("Primary index created successfully")
        except Exception as e:
            logger.warning(f"Error creating primary index: {e!s}")

        logger.info("Collection setup complete")
        return collection
    except Exception as e:
        raise RuntimeError(f"Error setting up collection: {e!s}")

# Setup collection
collection = setup_collection(
    cluster,
    os.environ["CB_BUCKET"],
    os.environ["SCOPE_NAME"],
    os.environ["COLLECTION_NAME"]
)

2025-07-07 12:14:42,840 - __main__ - INFO - Bucket 'vector-search-testing' exists
2025-07-07 12:14:45,858 - __main__ - INFO - Primary index created successfully
2025-07-07 12:14:45,858 - __main__ - INFO - Collection setup complete


### Vector Search Index Setup

Load the vector search index configuration from the existing agentcatalog_index.json file.

In [6]:
def setup_vector_search_index(cluster, index_definition):
    """Setup vector search index for flight data."""
    try:
        scope_index_manager = (
            cluster.bucket(os.environ["CB_BUCKET"]).scope(os.environ["SCOPE_NAME"]).search_indexes()
        )

        existing_indexes = scope_index_manager.get_all_indexes()
        index_name = index_definition["name"]

        if index_name not in [index.name for index in existing_indexes]:
            logger.info(f"Creating vector search index '{index_name}'...")
            search_index = SearchIndex.from_json(index_definition)
            scope_index_manager.upsert_index(search_index)
            logger.info(f"Vector search index '{index_name}' created successfully")
        else:
            logger.info(f"Vector search index '{index_name}' already exists")
    except Exception as e:
        raise RuntimeError(f"Error setting up vector search index: {e!s}")

# Load index definition from file (same as main.py)
try:
    with open("agentcatalog_index.json") as file:
        index_definition = json.load(file)
    logger.info("Loaded vector search index definition from agentcatalog_index.json")
    setup_vector_search_index(cluster, index_definition)
except Exception as e:
    logger.warning(f"Error loading index definition: {e!s}")
    logger.info("Continuing without vector search index...")

2025-07-07 12:14:45,866 - __main__ - INFO - Loaded vector search index definition from agentcatalog_index.json
2025-07-07 12:14:45,881 - __main__ - INFO - Vector search index 'vector_search_agentcatalog' already exists


## Section 3: Flight Data and Vector Store

Load our flight data and set up the vector store for semantic search.

In [9]:
def load_flight_data():
    """Load flight data from our enhanced flight_data.py file."""
    try:
        # Add the data directory to Python path
        sys.path.append("data")
        from flight_data import get_all_flight_data

        flight_data = get_all_flight_data()

        # Convert to text format for vector store
        flight_texts = []
        for item in flight_data:
            text = f"{item['title']} - {item['content']}"
            flight_texts.append(text)

        return flight_texts
    except Exception as e:
        raise ValueError(f"Error loading flight data: {e!s}")


def setup_vector_store(cluster):
    """Setup vector store and load flight data."""
    try:
        embeddings = OpenAIEmbeddings(
            api_key=os.environ["OPENAI_API_KEY"], model="text-embedding-3-small"
        )

        vector_store = CouchbaseSearchVectorStore(
            cluster=cluster,
            bucket_name=os.environ["CB_BUCKET"],
            scope_name=os.environ["SCOPE_NAME"],
            collection_name=os.environ["COLLECTION_NAME"],
            embedding=embeddings,
            index_name=os.environ["INDEX_NAME"],
        )

        flight_data = load_flight_data()

        try:
            vector_store.add_texts(texts=flight_data, batch_size=10)
            logger.info("Flight data loaded into vector store successfully")
        except Exception as e:
            logger.warning(
                f"Error loading flight data: {e!s}. Vector store created but data not loaded."
            )

        return vector_store
    except Exception as e:
        raise ValueError(f"Error setting up vector store: {e!s}")

# Setup vector store
vector_store = setup_vector_store(cluster)

2025-07-07 12:18:09,828 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'idempotency_key': 'stainless-python-retry-56dbcf3a-99a0-4011-9964-5e6fa9546f8c', 'post_parser': <function Embeddings.create.<locals>.parser at 0x10ea4c900>, 'json_data': {'input': [[46405, 356, 50322, 11216, 482, 27675, 36935, 10396, 13592, 555, 11989, 955, 323, 18912, 13, 38661, 14741, 26765, 2949, 220, 1187, 4207, 315, 22615, 5371, 2539, 73618, 13, 356, 14575, 811, 1903, 220, 22, 10, 2919, 1603, 25959, 62622, 264, 400, 1049, 11307, 369, 13018, 25952, 323, 400, 3443, 369, 6625, 25952, 13, 356, 14575, 811, 2949, 220, 22, 2919, 315, 25959, 50032, 275, 220, 1135, 4, 315, 11989, 907, 13, 26745, 323, 2626, 538, 14741, 3085, 810, 19303, 36935, 3878, 449, 11293, 12718, 13, 2360, 31637, 22961, 50032, 275, 4553, 11989, 907, 7389, 1327, 1509, 1113, 13463, 3881, 13], [9028, 894, 10539, 20287, 15292, 93075, 482, 63842, 10539, 18381, 2011, 539, 12771, 220, 1313, 87, 975

## Section 4: Agent Catalog Integration

Initialize the Agent Catalog and set up tool registration.

In [10]:
# Initialize Agent Catalog
catalog = agentc.Catalog(
    conn_string=os.environ["AGENT_CATALOG_CONN_STRING"],
    username=os.environ["AGENT_CATALOG_USERNAME"],
    password=os.environ["AGENT_CATALOG_PASSWORD"],
    bucket=os.environ["AGENT_CATALOG_BUCKET"],
)
application_span = catalog.Span(name="Flight Search Agent")

print("Agent Catalog initialized successfully")

2025-07-07 12:18:24,390 - agentc_core.config.config - DEBUG - Starting upwards search for the catalog folder in "/Users/kaustavghosh/Desktop/agent-catalog-quickstart/examples/flight_search_agent". Searching for ".agent-catalog".
2025-07-07 12:18:24,391 - agentc_core.config.config - DEBUG - Items in directory: [PosixPath('/Users/kaustavghosh/Desktop/agent-catalog-quickstart/examples/flight_search_agent/evals'), PosixPath('/Users/kaustavghosh/Desktop/agent-catalog-quickstart/examples/flight_search_agent/tools'), PosixPath('/Users/kaustavghosh/Desktop/agent-catalog-quickstart/examples/flight_search_agent/flight_search_agent_tutorial.ipynb'), PosixPath('/Users/kaustavghosh/Desktop/agent-catalog-quickstart/examples/flight_search_agent/.agent-catalog'), PosixPath('/Users/kaustavghosh/Desktop/agent-catalog-quickstart/examples/flight_search_agent/pyproject.toml'), PosixPath('/Users/kaustavghosh/Desktop/agent-catalog-quickstart/examples/flight_search_agent/tests'), PosixPath('/Users/kaustavghos

Agent Catalog initialized successfully


## Section 5: Parameter Mapping Implementation

This is a key feature of the flight search agent - intelligent parameter mapping using LLMs. This is copied directly from parameter_mapper.py.

In [11]:
# Import the exact ParameterMapper from parameter_mapper.py
from parameter_mapper import ParameterMapper

# Initialize chat model for parameter mapping
model_name = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
chat_model = langchain_openai.chat_models.ChatOpenAI(model=model_name, temperature=0.1)

# Initialize parameter mapper
parameter_mapper = ParameterMapper(chat_model)

print("Parameter mapper initialized")

# Test parameter mapping
test_text = "I want to fly from New York JFK to Los Angeles LAX"
airports = parameter_mapper.extract_airports_from_text(test_text)
print(f"\nParameter mapping test:")
print(f"Input: {test_text}")
print(f"Extracted: {airports}")

2025-07-07 12:18:32,361 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-68ac513a-d573-4a2f-9ea7-a85969afe039', 'json_data': {'messages': [{'content': '\n        Extract airport codes from text. Return JSON with source_airport and destination_airport.\n        Look for IATA codes (3 letters) or city names. Return null if not found.\n        ', 'role': 'system'}, {'content': 'Extract airports from: I want to fly from New York JFK to Los Angeles LAX', 'role': 'user'}], 'model': 'gpt-4o', 'stream': False, 'temperature': 0.1}}
2025-07-07 12:18:32,362 - openai._base_client - DEBUG - Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
2025-07-07 12:18:32,362 - httpcore.connection - DEBUG - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
2025-07-07 12:18:32,382 - httpcore.connection - DEBUG - connect_tcp.complete retu

Parameter mapper initialized


2025-07-07 12:18:34,398 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:48:34 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'704'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'711'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999931'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_0fe7d59500c31543a5da2340759364df'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=WMJP.0UAgiJynPxS5CdQb_RDuna96LYWoKxnzhmKgnM-1


Parameter mapping test:
Input: I want to fly from New York JFK to Los Angeles LAX
Extracted: {'source_airport': 'JFK', 'destination_airport': 'LAX'}


## Section 6: Flight Search Agent Implementation

This is the core FlightSearchAgent class and related state management, copied directly from main.py.

In [12]:
class FlightSearchState(agentc_langgraph.agent.State):
    """State for flight search conversations - single user system."""

    query: str
    resolved: bool
    search_results: list[dict]


class FlightSearchAgent(agentc_langgraph.agent.ReActAgent):
    """Flight search agent using Agent Catalog tools and prompts."""

    def __init__(self, catalog: agentc.Catalog, span: agentc.Span):
        """Initialize the flight search agent."""

        model_name = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
        chat_model = langchain_openai.chat_models.ChatOpenAI(model=model_name, temperature=0.1)

        # Initialize parameter mapper for intelligent parameter handling
        self.parameter_mapper = ParameterMapper(chat_model)

        super().__init__(
            chat_model=chat_model,
            catalog=catalog,
            span=span,
            prompt_name="flight_search_assistant",
        )

    def _invoke(
        self,
        span: agentc.Span,
        state: FlightSearchState,
        config: langchain_core.runnables.RunnableConfig,
    ) -> FlightSearchState:
        """Handle flight search conversation with Agent Catalog tools."""

        # Initialize conversation if this is the first message
        if not state["messages"]:
            initial_msg = langchain_core.messages.HumanMessage(content=state["query"])
            state["messages"].append(initial_msg)
            logger.info(f"Flight Query: {state['query']}")

        try:
            # Get tools from Agent Catalog using find method like hotel agent
            from langchain_core.tools import Tool

            tools = []
            tool_configs = [
                ("lookup_flight_info", "Find flight routes between airports"),
                ("save_flight_booking", "Save flight booking requests"),
                ("retrieve_flight_bookings", "Retrieve existing flight bookings"),
                ("search_flight_policies", "Search flight policies and guidelines"),
            ]

            for tool_name, description in tool_configs:
                try:
                    tool_obj = self.catalog.find("tool", name=tool_name)
                    if tool_obj and hasattr(tool_obj, "func"):
                        # Create a wrapper function using intelligent parameter mapping
                        def create_tool_wrapper(func, tool_name):
                            def wrapper(*args, **kwargs):
                                try:
                                    logger.debug(
                                        f"Tool {tool_name} called with args: {args}, kwargs: {kwargs}"
                                    )

                                    # Case 1: Handle ReAct agent format with multiple positional args
                                    if len(args) > 1:
                                        # ReAct agent passes multiple string arguments like ("SFO", "LAX")
                                        mapped_args = self.parameter_mapper.map_positional_args(
                                            tool_name, args, func
                                        )
                                        return func(**mapped_args)

                                    # Case 2: Handle single string argument (could be JSON or plain string)
                                    elif len(args) == 1 and isinstance(args[0], str) and not kwargs:
                                        input_str = args[0].strip()

                                        # Handle empty string or "None" for tools that take no args
                                        if not input_str or input_str.lower() in [
                                            "none",
                                            "null",
                                            "",
                                        ]:
                                            if tool_name == "retrieve_flight_bookings":
                                                return func()  # Call with no arguments
                                            else:
                                                # For other tools, try with empty dict
                                                mapped_args = (
                                                    self.parameter_mapper.map_parameters_smart(
                                                        tool_name, {}, func
                                                    )
                                                )
                                                return func(**mapped_args)

                                        # Try to parse as JSON first
                                        try:
                                            import json

                                            parsed_args = json.loads(input_str)

                                            if isinstance(parsed_args, dict):
                                                mapped_args = (
                                                    self.parameter_mapper.map_parameters_smart(
                                                        tool_name, parsed_args, func
                                                    )
                                                )
                                                return func(**mapped_args)
                                        except (json.JSONDecodeError, TypeError):
                                            pass  # Not JSON, continue to other parsing methods

                                        # Handle comma-separated format like "SFO", "LAX"
                                        if "," in input_str and '"' in input_str:
                                            try:
                                                # Parse comma-separated quoted strings
                                                import ast

                                                # Convert string like '"SFO", "LAX"' to tuple
                                                formatted_str = f"({input_str})"
                                                parsed_tuple = ast.literal_eval(formatted_str)
                                                if isinstance(parsed_tuple, tuple):
                                                    mapped_args = (
                                                        self.parameter_mapper.map_positional_args(
                                                            tool_name, parsed_tuple, func
                                                        )
                                                    )
                                                    return func(**mapped_args)
                                            except (ValueError, SyntaxError):
                                                pass

                                        # Fallback: treat as single parameter for search tools
                                        mapped_args = self.parameter_mapper.map_string_input(
                                            tool_name, input_str, func
                                        )
                                        return func(**mapped_args)

                                    # Case 3: Normal function call with kwargs
                                    elif kwargs:
                                        mapped_args = self.parameter_mapper.map_parameters_smart(
                                            tool_name, kwargs, func
                                        )
                                        return func(**mapped_args)

                                    # Case 4: No arguments - for tools like retrieve_flight_bookings
                                    elif len(args) == 0:
                                        if tool_name == "retrieve_flight_bookings":
                                            return func()
                                        else:
                                            mapped_args = (
                                                self.parameter_mapper.map_parameters_smart(
                                                    tool_name, {}, func
                                                )
                                            )
                                            return func(**mapped_args)

                                    # Default fallback
                                    return func(*args, **kwargs)

                                except Exception as e:
                                    logger.error(f"Tool {tool_name} execution error: {e}")
                                    logger.debug(f"Failed with args: {args}, kwargs: {kwargs}")
                                    return f"Error executing {tool_name}: {e!s}"

                            return wrapper

                        # Convert to LangChain Tool format with closure fix
                        wrapped_func = create_tool_wrapper(tool_obj.func, tool_name)
                        lc_tool = Tool(
                            name=tool_obj.meta.name,
                            description=tool_obj.meta.description or description,
                            func=wrapped_func,
                        )
                        tools.append(lc_tool)
                        logger.info(f"Loaded tool: {tool_name}")
                except Exception as e:
                    logger.warning(f"Could not load tool {tool_name}: {e}")

            if not tools:
                logger.error("No tools loaded from catalog")
                error_msg = langchain_core.messages.AIMessage(
                    content="I'm unable to access my flight search tools right now. Please try again later."
                )
                state["messages"].append(error_msg)
                state["resolved"] = True
                return state

            # Create a proper ReAct agent like hotel support agent
            from langchain import hub
            from langchain.agents import AgentExecutor, create_react_agent

            try:
                # Get ReAct prompt from hub
                react_prompt = hub.pull("hwchase17/react")

                # Create ReAct agent with tools
                agent = create_react_agent(self.chat_model, tools, react_prompt)
                agent_executor = AgentExecutor(
                    agent=agent,
                    tools=tools,
                    verbose=True,
                    handle_parsing_errors=True,
                    max_iterations=5,
                )

                # Execute the agent with the query
                query = state["query"]
                result = agent_executor.invoke({"input": query})

                # Create response message
                response_content = result.get(
                    "output", "I was unable to process your flight search request."
                )
                response_msg = langchain_core.messages.AIMessage(content=response_content)
                state["messages"].append(response_msg)

                logger.info(f"Agent Response: {response_content}")

                # Mark as resolved
                state["resolved"] = True

            except Exception as agent_error:
                logger.error(f"ReAct agent error: {agent_error}")
                # Fallback to simple model response
                fallback_response = self.chat_model.invoke(state["messages"])
                state["messages"].append(fallback_response)

                if hasattr(fallback_response, "content"):
                    logger.info(f"Fallback Response: {fallback_response.content}")

                state["resolved"] = True

        except Exception as e:
            logger.error(f"Agent invocation error: {e}")
            error_msg = langchain_core.messages.AIMessage(
                content=f"I encountered an error while processing your request: {e!s}"
            )
            state["messages"].append(error_msg)
            state["resolved"] = True

        return state

print("FlightSearchAgent class defined")

FlightSearchAgent class defined


## Section 7: LangGraph Workflow Implementation

This is the FlightSearchGraph class copied directly from main.py.

In [13]:
class FlightSearchGraph(agentc_langgraph.graph.GraphRunnable):
    """Flight search conversation graph using Agent Catalog."""

    @staticmethod
    def build_starting_state(query: str) -> FlightSearchState:
        """Build the initial state for the flight search - single user system."""
        return FlightSearchState(
            messages=[],
            query=query,
            resolved=False,
            search_results=[],
            previous_node=None,
        )

    def compile(self) -> langgraph.graph.graph.CompiledGraph:
        """Compile the LangGraph workflow."""

        # Build the flight search agent with catalog integration
        search_agent = FlightSearchAgent(catalog=self.catalog, span=self.span)

        # Create a simple workflow graph for flight search
        workflow = langgraph.graph.StateGraph(FlightSearchState)

        # Add the flight search agent node
        workflow.add_node("flight_search", search_agent)

        # Set entry point and simple flow
        workflow.set_entry_point("flight_search")
        workflow.add_edge("flight_search", langgraph.graph.END)

        return workflow.compile()

# Create the flight search graph
flight_graph = FlightSearchGraph(catalog=catalog, span=application_span)
# Compile the graph
compiled_graph = flight_graph.compile()

print("Flight search graph compiled successfully")

2025-07-07 12:18:38,888 - agentc_core.provider.loader - DEBUG - Handling entries with source /Users/kaustavghosh/Desktop/agent-catalog-quickstart/examples/flight_search_agent/tools/lookup_flight_info.py.
2025-07-07 12:18:38,889 - agentc_core.provider.loader - DEBUG - Loading module lookup_flight_info.
2025-07-07 12:18:38,976 - agentc_core.provider.loader - DEBUG - Handling entries with source /Users/kaustavghosh/Desktop/agent-catalog-quickstart/examples/flight_search_agent/tools/save_flight_booking.py.
2025-07-07 12:18:38,976 - agentc_core.provider.loader - DEBUG - Loading module save_flight_booking.
2025-07-07 12:18:39,009 - agentc_core.provider.loader - DEBUG - Handling entries with source /Users/kaustavghosh/Desktop/agent-catalog-quickstart/examples/flight_search_agent/tools/retrieve_flight_bookings.py.
2025-07-07 12:18:39,010 - agentc_core.provider.loader - DEBUG - Loading module retrieve_flight_bookings.
2025-07-07 12:18:39,046 - agentc_core.provider.loader - DEBUG - Handling entr

Flight search graph compiled successfully


## Section 8: Testing the Flight Search Agent

Now let's test our flight search agent with various scenarios.

In [14]:
def run_flight_search_query(query: str):
    """Run a flight search query and display results."""
    print(f"\nFlight Query: {query}")
    print("=" * 50)
    
    try:
        with application_span.new(f"Query: {query}") as span:
            logger.info(f"Flight Query: {query}")
            span["query"] = query

            # Build starting state - single user system
            state = FlightSearchGraph.build_starting_state(query=query)

            # Run the flight search
            result = compiled_graph.invoke(state)
            span["result"] = result

            # Display results summary
            if result.get("search_results"):
                logger.info(f"Found {len(result['search_results'])} flight options")

            logger.info(f"Search completed: {result.get('resolved', False)}")
            
            return result

    except Exception as e:
        logger.error(f"Search error: {e}")
        return None

print("Query function defined")

Query function defined


In [15]:
# Test query 1: Flight search
result = run_flight_search_query("Find flights from New York JFK to Los Angeles LAX")

2025-07-07 12:18:46,075 - agentc_core.activity.logger.base - DEBUG - Logging message: {
  "identifier": "53a33860d97b4c2390f7c5ac8e3f80b5",
  "span": {
    "session": "cd0ddc170ad04d24bc4776e71cb41753",
    "name": [
      "Flight Search Agent",
      "Query: Find flights from New York JFK to Los Angeles LAX"
    ]
  },
  "timestamp": "2025-07-07T12:18:46.065764+05:30",
  "content": {
    "kind": "begin"
  },
  "catalog_version": {
    "timestamp": "2025-07-04T11:20:31.313387Z",
    "identifier": "502f8f99816de30af7d6519116b62fd72933b54c",
    "is_dirty": false,
    "version_system": "git"
  }
}
2025-07-07 12:18:46,076 - __main__ - INFO - Flight Query: Find flights from New York JFK to Los Angeles LAX
2025-07-07 12:18:46,087 - agentc_core.activity.logger.base - DEBUG - Logging message: {
  "identifier": "113807afe7c747419f7b1273c46310a3",
  "span": {
    "session": "cd0ddc170ad04d24bc4776e71cb41753",
    "name": [
      "Flight Search Agent",
      "Query: Find flights from New York JF


Flight Query: Find flights from New York JFK to Los Angeles LAX


/Users/kaustavghosh/Desktop/agent-catalog-quickstart/.venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
2025-07-07 12:18:46,471 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): api.smith.langchain.com:443
2025-07-07 12:18:46,477 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (2): api.smith.langchain.com:443
2025-07-07 12:18:46,775 - urllib3.connectionpool - DEBUG - https://api.smith.langchain.com:443 "GET /info HTTP/1.1" 200 748
2025-07-07 12:18:46,778 - langsmith.client - DEBUG - Tracing control thread func compress parallel called
2025-07-07 12:18:46,796 - urllib3.connectionpool - DEBUG - https://api.smith.langchain.com:443 "GET /commits/hwchase17/react/latest HTTP/1.1" 200 933
2025-07-07 12:18:46,825 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'sta



> Entering new AgentExecutor chain...


2025-07-07 12:18:47,032 - langsmith.client - DEBUG - Client refs count indicates we're the only remaining reference to the client, stopping tracing thread
2025-07-07 12:18:47,033 - langsmith.client - DEBUG - Tracing control thread is shutting down
2025-07-07 12:18:47,583 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:48:47 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'377'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'383'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999666'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset

To find flights from New York JFK to Los Angeles LAX, I need to look up flight information between these two airports. 
Action: lookup_flight_info
Action Input: {"from": "JFK", "to": "LAX"}

2025-07-07 12:18:49,270 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:48:49 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'572'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'594'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999733'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_f08d416530aa49d95704af16a6e6f7af'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b

['AS flight from JFK to LAX using 321 762', 'B6 flight from JFK to LAX using 320', 'DL flight from JFK to LAX using 76W 752', 'QF flight from JFK to LAX using 744', 'AA flight from JFK to LAX using 32B 762', 'UA flight from JFK to LAX using 757', 'US flight from JFK to LAX using 32B 762', 'VX flight from JFK to LAX using 320']

2025-07-07 12:18:50,301 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:48:50 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'358'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'363'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999531'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_6cba79ebd532c677b2e2ef9e52be6616'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'

There are several flights available from New York JFK to Los Angeles LAX with different airlines and aircraft types. Here are the options:

1. AS flight using aircraft 321 or 762
2. B6 flight using aircraft 320
3. DL flight using aircraft 76W or 752
4. QF flight using aircraft 744
5. AA flight using aircraft 32B or 762
6. UA flight using aircraft 757
7. US flight using aircraft 32B or 762
8. VX flight using aircraft 320

Final Answer: There are multiple flights from New York JFK to Los Angeles LAX with airlines such as AS, B6, DL, QF, AA, UA, US, and VX, using various aircraft types.

> Finished chain.


In [16]:
# Test query 2: Policy search
result = run_flight_search_query("What's your baggage policy?")

2025-07-07 12:18:54,293 - agentc_core.activity.logger.base - DEBUG - Logging message: {
  "identifier": "4bb23ca48cac498296e4b95742c3fae0",
  "span": {
    "session": "cd0ddc170ad04d24bc4776e71cb41753",
    "name": [
      "Flight Search Agent",
      "Query: What's your baggage policy?"
    ]
  },
  "timestamp": "2025-07-07T12:18:54.273544+05:30",
  "content": {
    "kind": "begin"
  },
  "catalog_version": {
    "timestamp": "2025-07-04T11:20:31.313387Z",
    "identifier": "502f8f99816de30af7d6519116b62fd72933b54c",
    "is_dirty": false,
    "version_system": "git"
  }
}
2025-07-07 12:18:54,296 - __main__ - INFO - Flight Query: What's your baggage policy?
2025-07-07 12:18:54,303 - agentc_core.activity.logger.base - DEBUG - Logging message: {
  "identifier": "aac6f2ee395749c9b046d945620f3356",
  "span": {
    "session": "cd0ddc170ad04d24bc4776e71cb41753",
    "name": [
      "Flight Search Agent",
      "Query: What's your baggage policy?"
    ]
  },
  "timestamp": "2025-07-07T12:18:


Flight Query: What's your baggage policy?


2025-07-07 12:18:54,677 - urllib3.connectionpool - DEBUG - https://api.smith.langchain.com:443 "GET /info HTTP/1.1" 200 748
2025-07-07 12:18:54,678 - langsmith.client - DEBUG - Tracing control thread func compress parallel called
2025-07-07 12:18:54,700 - urllib3.connectionpool - DEBUG - https://api.smith.langchain.com:443 "GET /commits/hwchase17/react/latest HTTP/1.1" 200 933
2025-07-07 12:18:54,704 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-77cf315b-12da-4526-b90d-9fb9135e5c76', 'json_data': {'messages': [{'content': "Answer the following questions as best you can. You have access to the following tools:\n\nlookup_flight_info(*args, **kwargs) - Find flight routes between two airports with airline and aircraft information.\nsave_flight_booking(*args, **kwargs) - \nSave a flight booking to Couchbase database.\nHandles flight reservations with validation, pricing, and booking c



> Entering new AgentExecutor chain...


2025-07-07 12:18:54,933 - langsmith.client - DEBUG - Client refs count indicates we're the only remaining reference to the client, stopping tracing thread
2025-07-07 12:18:54,934 - langsmith.client - DEBUG - Tracing control thread is shutting down
2025-07-07 12:18:55,419 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:48:55 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'386'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'392'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999671'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset

...

Policy 1:
Carry-on Baggage Restrictions - Carry-on bags must not exceed 22x14x9 inches and 15 pounds. Personal items like purses, laptops, and small backpacks are permitted in addition to carry-on bags. Liquids must be in containers of 3.4oz or less and fit in a quart-sized clear bag. Prohibited items include sharp objects, flammable materials, and large electronics. Medical equipment and baby formula are exempt from liquid restrictions with proper documentation.

Policy 2:
Carry-on Baggage Restrictions - Carry-on bags must not exceed 22x14x9 inches and 15 pounds. Personal items like purses, laptops, and small backpacks are permitted in addition to carry-on bags. Liquids must be in containers of 3.4oz or less and fit in a quart-sized clear bag. Prohibited items include sharp objects, flammable materials, and large electronics. Medical equipment and baby formula are exempt from liquid restrictions with proper documentation.

Policy 3:
Carry-on Baggage Restrictions - Carry-on bags must n

2025-07-07 12:19:17,260 - httpcore.connection - DEBUG - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
2025-07-07 12:19:17,298 - httpcore.connection - DEBUG - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x119d86660>
2025-07-07 12:19:17,299 - httpcore.connection - DEBUG - start_tls.started ssl_context=<ssl.SSLContext object at 0x10fa33bd0> server_hostname='api.openai.com' timeout=None
2025-07-07 12:19:17,327 - httpcore.connection - DEBUG - start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x119bc6cf0>
2025-07-07 12:19:17,328 - httpcore.http11 - DEBUG - send_request_headers.started request=<Request [b'POST']>
2025-07-07 12:19:17,328 - httpcore.http11 - DEBUG - send_request_headers.complete
2025-07-07 12:19:17,328 - httpcore.http11 - DEBUG - send_request_body.started request=<Request [b'POST']>
2025-07-07 12:19:17,329 - httpcore.http11 - DEBUG - send_request_body.complet

The baggage policy is as follows: Carry-on bags must not exceed dimensions of 22x14x9 inches and a weight of 15 pounds. In addition to carry-on bags, personal items such as purses, laptops, and small backpacks are allowed. Liquids must be in containers of 3.4 ounces or less and fit within a quart-sized clear bag. Prohibited items include sharp objects, flammable materials, and large electronics. However, medical equipment and baby formula are exempt from liquid restrictions if proper documentation is provided.Invalid Format: Missing 'Action:' after 'Thought:'

2025-07-07 12:19:20,369 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:49:20 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'310'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'314'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999128'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'1ms'), (b'x-request-id', b'req_e7f16f4e8290c157a37b273ac268bc1a'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b

I now know the final answer.

Final Answer: The baggage policy is as follows: Carry-on bags must not exceed dimensions of 22x14x9 inches and a weight of 15 pounds. In addition to carry-on bags, personal items such as purses, laptops, and small backpacks are allowed. Liquids must be in containers of 3.4 ounces or less and fit within a quart-sized clear bag. Prohibited items include sharp objects, flammable materials, and large electronics. However, medical equipment and baby formula are exempt from liquid restrictions if proper documentation is provided.

> Finished chain.


In [17]:
# Test query 3: Booking retrieval
result = run_flight_search_query("Show me my current bookings")

2025-07-07 12:19:28,477 - agentc_core.activity.logger.base - DEBUG - Logging message: {
  "identifier": "aefe85d536ab48c0a70d996267d80965",
  "span": {
    "session": "cd0ddc170ad04d24bc4776e71cb41753",
    "name": [
      "Flight Search Agent",
      "Query: Show me my current bookings"
    ]
  },
  "timestamp": "2025-07-07T12:19:28.474770+05:30",
  "content": {
    "kind": "begin"
  },
  "catalog_version": {
    "timestamp": "2025-07-04T11:20:31.313387Z",
    "identifier": "502f8f99816de30af7d6519116b62fd72933b54c",
    "is_dirty": false,
    "version_system": "git"
  }
}
2025-07-07 12:19:28,478 - __main__ - INFO - Flight Query: Show me my current bookings
2025-07-07 12:19:28,485 - agentc_core.activity.logger.base - DEBUG - Logging message: {
  "identifier": "e29026f8c35540da84ec8eda3fcae4bd",
  "span": {
    "session": "cd0ddc170ad04d24bc4776e71cb41753",
    "name": [
      "Flight Search Agent",
      "Query: Show me my current bookings"
    ]
  },
  "timestamp": "2025-07-07T12:19:


Flight Query: Show me my current bookings


2025-07-07 12:19:28,884 - urllib3.connectionpool - DEBUG - https://api.smith.langchain.com:443 "GET /info HTTP/1.1" 200 748
2025-07-07 12:19:28,885 - langsmith.client - DEBUG - Tracing control thread func compress parallel called
2025-07-07 12:19:28,933 - urllib3.connectionpool - DEBUG - https://api.smith.langchain.com:443 "GET /commits/hwchase17/react/latest HTTP/1.1" 200 933
2025-07-07 12:19:28,936 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-d6cb2709-921c-4208-a3fc-cee1c4be0541', 'json_data': {'messages': [{'content': 'Answer the following questions as best you can. You have access to the following tools:\n\nlookup_flight_info(*args, **kwargs) - Find flight routes between two airports with airline and aircraft information.\nsave_flight_booking(*args, **kwargs) - \nSave a flight booking to Couchbase database.\nHandles flight reservations with validation, pricing, and booking c



> Entering new AgentExecutor chain...


2025-07-07 12:19:29,140 - langsmith.client - DEBUG - Client refs count indicates we're the only remaining reference to the client, stopping tracing thread
2025-07-07 12:19:29,141 - langsmith.client - DEBUG - Tracing control thread is shutting down
2025-07-07 12:19:29,913 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:49:29 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'376'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'385'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999671'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset

To show the current bookings, I need to retrieve all flight bookings from the database.
Action: retrieve_flight_bookings
Action Input: NoneYour Current Bookings (1 found):

Booking 1:
  Booking ID: FL0708CD66805E
  Route: SFO → LAX
  Date: 2025-07-08
  Passengers: 1
  Class: Economy
  Total: $250.00
  Status: confirmed
  Booked: 2025-07-07

2025-07-07 12:19:31,153 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:49:31 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'386'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'391'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999580'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_80dd10c57d285023d5050357c0aaaf1c'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'

I now know the final answer.

Final Answer: You have one current booking:
- Booking ID: FL0708CD66805E
- Route: SFO → LAX
- Date: 2025-07-08
- Passengers: 1
- Class: Economy
- Total: $250.00
- Status: confirmed
- Booked on: 2025-07-07

> Finished chain.


In [18]:
# Test query 4: Flight booking
result = run_flight_search_query("Book a flight from San Francisco to Chicago for tomorrow")

2025-07-07 12:19:35,999 - agentc_core.activity.logger.base - DEBUG - Logging message: {
  "identifier": "c5b0525254bf49d6a8fb7ae17b4c1af4",
  "span": {
    "session": "cd0ddc170ad04d24bc4776e71cb41753",
    "name": [
      "Flight Search Agent",
      "Query: Book a flight from San Francisco to Chicago for tomorrow"
    ]
  },
  "timestamp": "2025-07-07T12:19:35.994554+05:30",
  "content": {
    "kind": "begin"
  },
  "catalog_version": {
    "timestamp": "2025-07-04T11:20:31.313387Z",
    "identifier": "502f8f99816de30af7d6519116b62fd72933b54c",
    "is_dirty": false,
    "version_system": "git"
  }
}
2025-07-07 12:19:35,999 - __main__ - INFO - Flight Query: Book a flight from San Francisco to Chicago for tomorrow
2025-07-07 12:19:36,006 - agentc_core.activity.logger.base - DEBUG - Logging message: {
  "identifier": "6357c75c37de41cda6a4127003498255",
  "span": {
    "session": "cd0ddc170ad04d24bc4776e71cb41753",
    "name": [
      "Flight Search Agent",
      "Query: Book a flight f


Flight Query: Book a flight from San Francisco to Chicago for tomorrow


2025-07-07 12:19:36,339 - urllib3.connectionpool - DEBUG - https://api.smith.langchain.com:443 "GET /info HTTP/1.1" 200 748
2025-07-07 12:19:36,340 - langsmith.client - DEBUG - Tracing control thread func compress parallel called
2025-07-07 12:19:36,363 - urllib3.connectionpool - DEBUG - https://api.smith.langchain.com:443 "GET /commits/hwchase17/react/latest HTTP/1.1" 200 933
2025-07-07 12:19:36,369 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-f254d507-27c0-455a-a49c-7a0844a6462a', 'json_data': {'messages': [{'content': 'Answer the following questions as best you can. You have access to the following tools:\n\nlookup_flight_info(*args, **kwargs) - Find flight routes between two airports with airline and aircraft information.\nsave_flight_booking(*args, **kwargs) - \nSave a flight booking to Couchbase database.\nHandles flight reservations with validation, pricing, and booking c



> Entering new AgentExecutor chain...


2025-07-07 12:19:36,593 - langsmith.client - DEBUG - Client refs count indicates we're the only remaining reference to the client, stopping tracing thread
2025-07-07 12:19:36,594 - langsmith.client - DEBUG - Tracing control thread is shutting down
2025-07-07 12:19:38,274 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:49:38 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'864'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'867'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999665'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset

...

To proceed with booking a flight, I need to select one of the available flights. Let's choose the United Airlines (UA) flight, as it offers multiple aircraft options, which might provide more flexibility.

Action: save_flight_booking
Action Input: {"airline": "UA", "from": "SFO", "to": "ORD", "date": "tomorrow", "aircraft": "738"}

2025-07-07 12:19:42,973 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:49:43 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'839'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'843'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999702'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_6073f2f34435e4b2496877b4f38f05ec'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b

Flight Booking Confirmed!

Booking ID: FL0708F5C33C1E
Route: SFO → ORD
Departure Date: 2025-07-08
Return Date: One-way
Passengers: 1
Class: Economy
Total Price: $250.00

Next Steps:
1. Check-in opens 24 hours before departure
2. Arrive at airport 2 hours early for domestic flights
3. Bring valid government-issued photo ID
4. Booking confirmation sent to your email

Thank you for choosing our airline!

2025-07-07 12:19:43,961 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:49:43 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'348'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'351'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999356'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'1ms'), (b'x-request-id', b'req_11fb2fbe2056132ade86a4ff9cdb9e1f'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b

I now know the final answer.

Final Answer: The flight from San Francisco (SFO) to Chicago (ORD) for tomorrow has been successfully booked with United Airlines (UA) using aircraft 738. The booking ID is FL0708F5C33C1E, and the total price is $250.00. Please remember to check in 24 hours before departure and arrive at the airport 2 hours early with a valid government-issued photo ID. A booking confirmation has been sent to your email.

> Finished chain.


In [19]:
# Test query 5: Cancellation policy
result = run_flight_search_query("What are the cancellation rules?")

2025-07-07 12:19:49,709 - agentc_core.activity.logger.base - DEBUG - Logging message: {
  "identifier": "2031454bda184e65be271386a1216213",
  "span": {
    "session": "cd0ddc170ad04d24bc4776e71cb41753",
    "name": [
      "Flight Search Agent",
      "Query: What are the cancellation rules?"
    ]
  },
  "timestamp": "2025-07-07T12:19:49.705397+05:30",
  "content": {
    "kind": "begin"
  },
  "catalog_version": {
    "timestamp": "2025-07-04T11:20:31.313387Z",
    "identifier": "502f8f99816de30af7d6519116b62fd72933b54c",
    "is_dirty": false,
    "version_system": "git"
  }
}
2025-07-07 12:19:49,709 - __main__ - INFO - Flight Query: What are the cancellation rules?
2025-07-07 12:19:49,713 - agentc_core.activity.logger.base - DEBUG - Logging message: {
  "identifier": "7f7544c54a174bdea0d49ae128154e7e",
  "span": {
    "session": "cd0ddc170ad04d24bc4776e71cb41753",
    "name": [
      "Flight Search Agent",
      "Query: What are the cancellation rules?"
    ]
  },
  "timestamp": "20


Flight Query: What are the cancellation rules?


2025-07-07 12:19:50,048 - urllib3.connectionpool - DEBUG - https://api.smith.langchain.com:443 "GET /info HTTP/1.1" 200 748
2025-07-07 12:19:50,050 - langsmith.client - DEBUG - Tracing control thread func compress parallel called
2025-07-07 12:19:50,070 - urllib3.connectionpool - DEBUG - https://api.smith.langchain.com:443 "GET /commits/hwchase17/react/latest HTTP/1.1" 200 933
2025-07-07 12:19:50,133 - langsmith.client - DEBUG - Client refs count indicates we're the only remaining reference to the client, stopping compression thread
2025-07-07 12:19:50,133 - langsmith.client - DEBUG - Closing Client.session
2025-07-07 12:19:50,134 - langsmith.client - DEBUG - Compressed traces control thread is shutting down
2025-07-07 12:19:50,138 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-5c06cf28-c38e-42a9-8814-b79fa20ed399', 'json_data': {'messages': [{'content': 'Answer the following ques



> Entering new AgentExecutor chain...


2025-07-07 12:19:50,304 - langsmith.client - DEBUG - Client refs count indicates we're the only remaining reference to the client, stopping tracing thread
2025-07-07 12:19:50,305 - langsmith.client - DEBUG - Tracing control thread is shutting down
2025-07-07 12:19:50,982 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:49:51 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'353'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'356'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999671'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset

...

The cancellation rules for flights are as follows:

- Economy tickets cancelled within 24 hours of booking receive full refunds.
- Cancellations made 7 or more days before departure incur a $200 fee for domestic flights and a $400 fee for international flights.
- Cancellations within 7 days of departure forfeit 50% of the ticket value.
- Premium and business class tickets offer more flexible cancellation terms with reduced fees.
- No-show passengers forfeit the entire ticket value unless extenuating circumstances apply.

These policies are consistent across the retrieved policies.Invalid Format: Missing 'Action:' after 'Thought:'

2025-07-07 12:19:57,693 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:49:57 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'341'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'345'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29998875'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'2ms'), (b'x-request-id', b'req_392156121251670f41636adcd15f976e'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b

The cancellation rules for flights are as follows:

- Economy tickets cancelled within 24 hours of booking receive full refunds.
- Cancellations made 7 or more days before departure incur a $200 fee for domestic flights and a $400 fee for international flights.
- Cancellations within 7 days of departure forfeit 50% of the ticket value.
- Premium and business class tickets offer more flexible cancellation terms with reduced fees.
- No-show passengers forfeit the entire ticket value unless extenuating circumstances apply.

These policies are consistent across the retrieved policies.Invalid Format: Missing 'Action:' after 'Thought:'

2025-07-07 12:19:59,193 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 07 Jul 2025 06:49:59 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'couchbase-q6vk81'), (b'openai-processing-ms', b'423'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'428'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29998711'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'2ms'), (b'x-request-id', b'req_26708184160e32441af95890ebab8c77'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b

I now know the final answer.

Final Answer: The cancellation rules for flights are as follows:

- Economy tickets cancelled within 24 hours of booking receive full refunds.
- Cancellations made 7 or more days before departure incur a $200 fee for domestic flights and a $400 fee for international flights.
- Cancellations within 7 days of departure forfeit 50% of the ticket value.
- Premium and business class tickets offer more flexible cancellation terms with reduced fees.
- No-show passengers forfeit the entire ticket value unless extenuating circumstances apply.

> Finished chain.


## Conclusion

You've successfully built a flight search agent using:

### What We Accomplished
- **Infrastructure Setup**: Configured Couchbase with vector search capabilities
- **Agent Catalog Integration**: Registered and utilized intelligent tools
- **Smart Parameter Mapping**: Implemented LLM-powered parameter extraction
- **LangGraph Agent**: Built a ReAct agent with sophisticated workflows
- **Vector Search**: Enabled semantic policy and information retrieval

### Key Features Demonstrated
1. **Flight Information Lookup** - Search routes between airports
2. **Intelligent Booking System** - Create bookings with validation
3. **Booking Management** - Retrieve and manage existing bookings
4. **Policy Search** - Semantic search through flight policies
5. **Smart Parameter Handling** - Flexible input processing

### Further Learning
- [Agent Catalog Documentation](https://docs.agentcatalog.ai)
- [Couchbase Vector Search Guide](https://docs.couchbase.com/server/current/vector-search/vector-search.html)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)

Thank you for completing this tutorial!